## Library Imports

In [28]:
#connect to google sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

#google translate
!pip install googletrans
from googletrans import Translator
tr = Translator()

import pandas as pd

# !pip install requests
# !pip install beautifulsoup4

import requests
import urllib.request
import time
from bs4 import BeautifulSoup

#parallel processing
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

import timeit

from IPython.display import clear_output
clear_output()

# Inputs

In [33]:
# @markdown This cell connects to the output sheet

output_url = 'https://docs.google.com/spreadsheets/d/1pYvN_TTqxG0uU1AgubTWJo0LZbEDTPY-zRSxxaCAtHE/' #@param {type:"string"}
output_ws = 'testing'
output_row = 2


In [30]:
#maia
maia_url = 'https://docs.google.com/spreadsheets/d/1zit8jJhj7ccbX9P4iiXaW3Fxs5ITrEaZ6UZCw9isEAQ/'            #id of spreadsheet with list of objects (now MAIA)
maia_ws = 'DB_KeyProd'                    #worksheet with products
maia_row = 5

wb1 = gc.open_by_url(maia_url)
sh1 = wb1.worksheet(maia_ws)
maia_df = pd.DataFrame(sh1.get_all_values(),columns=sh1.row_values(maia_row)).iloc[maia_row:].reset_index(drop=True)

#make df with key products, keywords
df_key = maia_df[["Key Product", "Keywords", "Category"]].reset_index(drop=True)
df_key.columns = ['key_product','keywords', 'keyword_count', 'maia_category']
df_key = df_key.drop(['keyword_count'], axis=1)

# Functions

## Scrapers

In [38]:
def bol_cat(prod):
  start = time.time()
  s = time.time()
  URL = "https://www.bol.com/nl/s/?searchtext="+prod+"&searchContext=media_all&appliedSearchContextId=media_all&suggestFragment=&adjustedSection=&originalSection=&originalSearchContext=&section=main&N=0&defaultSearchContext=media_all"
  response = requests.get(URL)
  e = time.time()
  # print(e-s)
  s = time.time()
  soup = BeautifulSoup(response.text, 'html.parser')
  e = time.time()
  # print(e-s)

  try:
    s = time.time()
    # find url extension (href) of first product in search results
    soup = soup.find('div',{'class':'product-item__info hit-area'})
    alist = soup.findAll('a')[1]
    href = alist['href']
    e = time.time()
    # print(e-s)


    s = time.time()
    # open page of first product
    URL = 'https://www.bol.com'+href
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    e = time.time()
    # print(e-s)

    soup = soup.findAll('li', {'class':'breadcrumbs__item'})
    s4 = str(soup[2].find('a'))
    s5 = str(soup[3].find('a'))

    # this is bad:
    # get categories
    s = time.time()
    s4 = s4[s4.find('name">')+6:s4.find('</span>')]
    s5 = s5[s5.find('name">')+6:s5.find('</span>')]
    e = time.time()
    # print(e-s)
    # convert categories to single string
    cat = s4+' // '+s5+' // '
  except:
    cat = 'Unknown'
  end = time.time()
  # print(end-start)
  return cat

def cb_cat(prod):
  URL = "https://www.coolblue.nl/zoeken?query="+prod
  response = requests.get(URL)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    # find url extension (href) of first product in search results
    soup = soup.find('div',{'class':'product-grid__card'})
    alist = soup.findAll('a')[1]
    href = alist['href']

    # open page of first product
    URL = 'https://www.coolblue.nl'+href
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # get categories
    soup = soup.findAll('ol', {'class':'breadcrumbs is-visible breadcrumbs--has-bottom-spacing'})
    alist = soup[0].findAll('a', {'class':'link'})
    # convert categories to single string
    cat = ''
    for i in alist[1:]:
      cat = cat + i['title'] + ' // '
  except:
    cat = 'Unknown'
  return cat

def ah_cat(prod):
  URL = "https://www.ah.nl/zoeken?query="+prod
  response = requests.get(URL)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    # find url extension (href) of first product in search result
    soup = soup.find('div',{'class':'product-card-portrait_content__KKpmv'})
    alist = soup.findAll('a')
    href = alist[0]['href']
    # open page of first product
    URL = 'https://www.ah.nl'+href
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    # get categories
    soup = soup.findAll('ol', {'class':'breadcrumbs_list__1g8sQ page-navigation_breadcrumbs__22sFV'})
    s3 = soup[0].findAll('a', {'class':'breadcrumbs_link__2Edfx'})
    cat = ''
    # convert categories to single string
    for i in range(len(s3[2:])):
      cat = cat+s3[i+2]['title']+' // '
  except:
    cat = 'Unknown'
  return cat


## Apply's

In [9]:
def bol_apply(x):
  search = x['key_product']
  search = tr.translate(search, dest='nl').text
  search = search.replace(' ', '+')
  x['bol_category'] = bol_cat(search)
  return x

def cb_apply(x):
  search = x['key_product'].replace(' ', '+')
  if x['maia_category'] == 'ICT':
    x['cb_category'] = cb_cat(x['key_product'])
  else:
    x['cb_category'] = ' '
  return x

def ah_apply(x):
  search = x['key_product']
  search = tr.translate(search, dest='nl').text
  search = x['key_product'].replace(' ', '%20')
  if x['maia_category'] == 'Food':
    x['ah_category'] = ah_cat(x['key_product'])
  elif x['maia_category'] == 'Drinks':
    x['ah_category'] = ah_cat(x['key_product'])
  else:
    x['ah_category'] = ' '
  return x

## Write to spreadsheet function

In [10]:
def output_to_ws(data, column, ws, row):
  #data: df column
  #column: string of column index; 'A', 'B', 'C' etc.
  #ws: worksheet of interest
  
  cell_list = ws.range(column+str(row)+':'+column+str(len(data)+row-1))
  for i in range(len(cell_list)):
    try:
      cell_list[i].value = float(data[i])
    except:
      cell_list[i].value = data[i]
  ws.update_cells(cell_list)
  return column

# Main

In [36]:
# bol guesses
start = time.time()
# df_key = df_key.apply(bol_apply, axis=1)
df_key = df_key.parallel_apply(bol_apply, axis=1)
end = time.time()
print("Bol time: %s sec" % round(end-start, 2))

# coolblue guesses
start = time.time()
# df_key = df_key.apply(cb_apply, axis=1)
df_key = df_key.parallel_apply(cb_apply, axis=1)
end = time.time()
print("Coolblue time: %s sec" % round(end-start, 2))

# albert heijn guesses
start = time.time()
# df_key = df_key.apply(ah_apply, axis=1)
df_key = df_key.parallel_apply(ah_apply, axis=1)
end = time.time()
print("Albert Heijn time: %s sec" % round(end-start, 2))

Bol time: 11.25 sec
Coolblue time: 0.1 sec
Albert Heijn time: 13.99 sec


In [37]:
# write to spreadsheet
#output column names
cols = ['A', 'B', 'C', 'D', 'E', 'F']
#columns of data to output
datasets = [df_key['key_product'], df_key['keywords'], df_key['maia_category'], df_key['bol_category'], df_key['cb_category'], df_key['ah_category']]
output_ws = gc.open_by_url(output_url).worksheet(output_ws)
for i in range(len(datasets)):
  output_to_ws(datasets[i], cols[i], output_ws, output_row)

# Archive

In [ ]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

#input url of search page for product
prod = 'chocolate'
url = 'https://www.walmart.com/search/?query='+prod
response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)
soup = soup.find('div',{'class':'search-product-result'})
print(soup)
alist = soup.findAll('a')

def get_name(href):
    url = 'https://www.walmart.com/'+href
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    soup = soup.find('h1',{'class':'prod-ProductTitle prod-productTitle-buyBox font-bold'})
    return soup['content']

laptops=[]

for i in range(len(alist)):
    laptops.append(get_name(alist[i]['href']))

In [ ]:
!pip install -U selenium

from selenium import webdriver as driver

browser = driver.PhantomJS()
p = browser.get("http://en.wikipedia.org/wiki/StackOverflow")
assert "Stack Overflow - Wikipedia" in browser.title